In [1]:
import xarray as xr
import dask
import math
import numpy as np

/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
Training_Images = np.load('/fast/gmooers/Preprocessed_Data/W_Trial/W_Training.npy')
Test_Images = np.load('/fast/gmooers/Preprocessed_Data/W_Trial/W_Test.npy')

In [3]:
import keras
from keras import layers
from keras import backend as K
from keras.models import Model

Using TensorFlow backend.


In [19]:
input_dim = (len(Training_Images[0]), len(Training_Images[0][0]),1)
encoder_input = layers.Input(input_dim, name = 'encoder_input')
x = encoder_input
dim1 = 64
dim2 = 3
n_layers_encoder = 4
stride = 2
zdim = 32
acti = "selu"

x = layers.Conv2D(dim1, dim2, padding='same', activation = acti)(encoder_input)
x = layers.Conv2D(dim1, dim2, padding='same', activation = acti, strides = (stride, stride))(x)
x = layers.Conv2D(dim1, dim2, padding='same', activation = acti)(x)
x = layers.Conv2D(dim1, dim2, padding='same', activation = acti)(x)
shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)

mu = layers.Dense(zdim, name='mu')(x)
log_var = layers.Dense(zdim, name = 'log_var')(x)

encoder_mu_log_var = Model(encoder_input, (mu, log_var))





In [20]:
def sampling(args):
    mu, log_var = args
    epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
    return mu+K.exp(log_var/2)*epsilon

In [22]:
encoder_output = layers.Lambda(sampling, name='encoder_output')([mu, log_var])

In [23]:
encoder = Model(encoder_input, encoder_output)

In [ ]:
dim1 = 32
dim2 = 3
small_dim = 1
stride = 2
acti = 'relu'
acti_function = 'sigmoid'

In [ ]:
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation=acti)(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(dim1, dim2, padding='same', activation=acti, strides=(stride, stride))(x)
x = layers.Conv2D(small_dim, dim2, padding='same', activation=acti_function)(x)
decoder = Model(decoder_input, x)
z_decoded = decoder(z)

In [26]:
#given in example
learning_rate = 0.0005
optimizer = keras.optimizers.Adam(lr=learning_rate)

In [27]:
def vae_r_loss(y_true, y_pred):
    r_loss = K.mean(K.square(y_true-y_pred), axis = [1,2,3])
    return r_loss_factor*r_loss

def vae_kl_loss(y_true, y_pred):
    kl_loss = -0.5*K.sum(1+self.log_var-K.square(self.mu)-K.exp(self.log_var), axis = 1)
    return kl_loss

def vae_loss(y_true, y_pred):
    r_loss = vae_r_loss(y_true, y_pred)
    kl_loss = vae_kl_loss(y_true, y_pred)
    return r_loss+kl_loss


In [32]:

Model.compile(optimizer=optimizer, loss=vae_loss, metrics = [vae_r_loss, vae_kl_loss])

TypeError: compile() missing 1 required positional argument: 'self'

In [ ]:
n_to_show = 30
z_new = np.random.normal(size = (n_to_show, VAE.zdim))
print(z_new)